# Convolution neural network

In [1]:
import pandas as pd, numpy as np, os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # settings like template by default

from dotenv import load_dotenv
from pathlib import Path

load_dotenv(dotenv_path = Path('.') / '.config')

pio.templates.default = os.getenv("template_plotly") # default template

from helper_functs.tsfuncts import create_features_extracted_days, calculate_metrics, df_average_error, split_sequence

import tensorflow as tf

only_cpu = True
if only_cpu: 
    os.environ["CUDA_VISIBLE_DEVICES"] = str(os.getenv("CUDA_VISIBLE_DEVICES"))
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(os.getenv("TF_CPP_MIN_LOG_LEVEL"))  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR
else:
    gpus = tf.config.list_physical_devices('GPU')  # obtiene las gpus instaladas
    if len(gpus) > 0:   # If GPU available
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(gpus)

2025-06-06 13:46:55.496247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749217615.741342     475 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749217615.772660     475 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Read data
df = pd.read_csv('data/airline-passengers.csv', header=0, index_col=0, parse_dates=True)
print(f"Size data: {df.shape} min date: {min(df.index)} and max date: {max(df.index)}")
print(df.head(3))

Size data: (144, 1) min date: 1949-01-01 00:00:00 and max date: 1960-12-01 00:00:00
            Passengers
Month                 
1949-01-01         112
1949-02-01         118
1949-03-01         132


In [11]:
# number of steps
date_split = "1959-01-01"
n_steps, n_features = 12, 1
# train_date = pd.to_datetime(date_str) + pd.Timedelta(months=12)
# train_date = pd.to_datetime(date_str) + pd.Timedelta(hours=12)
train_date = pd.to_datetime(date_split) + pd.DateOffset(months=n_steps-1)
compared_date = pd.to_datetime(train_date) + pd.DateOffset(months=n_features)
train, test = df.loc[:train_date], df.loc[date_split:] # test set starts before the end of the train set
print(f"Train set: {train.shape} and Test set: {test.shape}")
print(f"Train maz date: {max(train.index)} and test min date {min(test.index)}")

Train set: (132, 1) and Test set: (24, 1)
Train maz date: 1959-12-01 00:00:00 and test min date 1959-01-01 00:00:00


In [12]:
# Variables used in deep learning models
my_batch, my_epochs, my_val = 24, 200, 0.12
print(f"n epochs {my_epochs}")
# early_stop helps to stop training when using GridSearchCV for deep learning models
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',     # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
    patience=12,             # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value
)

n epochs 200


## Univariate series, one output (One step)

In [13]:
%%time
# Generate train sample
X_train, y_train = split_sequence(train.values, n_steps, n_features)
print(f"train shape: {X_train.shape, y_train.shape}")
# Generate test sample
X_test, y_test = split_sequence(test.values, n_steps, n_features)
print(f"test shape: {X_test.shape, y_test.shape}")

# Show the first sample
for i in range(1):
    print(X_train[i].tolist(), y_train[i])
    print(X_test[i].tolist(), y_test[i])

train shape: ((120, 12, 1), (120, 1))
test shape: ((12, 12, 1), (12, 1))
[[112], [118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118]] [115]
[[360], [342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405]] [417]
CPU times: user 669 μs, sys: 0 ns, total: 669 μs
Wall time: 640 μs


In [14]:
%%time
# Define a CNN model
model1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_steps, n_features)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu'), # 64 filtros de tamaño 2
    tf.keras.layers.MaxPooling1D(pool_size=2), # maxpooling size = 2
    tf.keras.layers.Flatten(), # flaten layer
    tf.keras.layers.Dense(50, activation='relu'), # dense layer
    tf.keras.layers.Dense(1) # one output neuron
])

model1.compile(optimizer='adam', loss='mse')
# Train data
model1.fit(X_train, y_train, epochs=my_epochs, verbose=False, 
            validation_split=my_val, batch_size=my_batch, callbacks=[early_stop])

best_epoch = np.argmin(model1.history.history['val_loss']) + 1
fig = px.line(pd.DataFrame(model1.history.history), y='loss', title=f'Loss til epoch {best_epoch}')
fig.show()

CPU times: user 12.9 s, sys: 3.08 s, total: 16 s
Wall time: 13 s


In [15]:
updated_array = X_test[0:1, :, :]
list_predicted = []
for index in range(X_test.shape[0]):
    yhat = model1.predict(updated_array, verbose=False)  # predice el punto siguiente
    list_predicted.append(yhat[0, 0].tolist())
    # print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 1:, :], yhat.reshape(1, 1, 1)), axis=1)
print(f"Predicted values: {len(list_predicted)}")
print(f"y_test shape {y_test.shape}")
# rmse = np.sqrt(np.mean((y_test - np.array(list_predicted))**2))
score = calculate_metrics(y_test, np.array(list_predicted))
print("Score:", score)

Predicted values: 12
y_test shape (12, 1)
Score: {'RMSE': np.float64(24.61), 'MAE': 20.65}


In [16]:
compared_df = test[compared_date:].copy()
compared_df['Predicted'] = list_predicted
print(f"Results shape {compared_df.shape}")
print(compared_df.head(2))
# rmse = np.sqrt(np.mean((compared_df.Passengers - compared_df.Predicted)**2))
score = calculate_metrics(compared_df.Passengers, compared_df.Predicted)
fig = px.line(compared_df, x = compared_df.index, y= ['Passengers', 'Predicted'],
            labels = {'value': 'Number of passengers', 'variable': 'Legend'},
            title = f'Prediction with a CNN model (one step) score: {score}')
fig.show()

Results shape (12, 2)
            Passengers   Predicted
Month                             
1960-01-01         417  390.249939
1960-02-01         391  400.921967


## Univariate series, one output (Multi step)

In [12]:
def split_sequence(sequence: list, n_steps_in: int, n_steps_out: int, n_features: int):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in 
        out_end_ix = end_ix + n_steps_out
        
        if out_end_ix > len(sequence):
            break
        
        X.append( sequence[i:end_ix] ) 
        y.append( sequence[end_ix:out_end_ix] )
    
    X = np.array(X)
    # reshape from [samples, timesteps] to [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    y = np.array(y)
    return X, y

# choose the number of steps for input and output
n_steps_in, n_steps_out, n_features = 12, 3, 1

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out, n_features)
print(X.shape, y.shape)
# show the shape of the generated samples
for i in range(2):
    print(X[i].tolist(), y[i])

(118, 12, 1) (118, 3)
[[112], [118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118]] [115 126 141]
[[118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118], [115]] [126 141 135]


In [13]:
# Define a CNN model
model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_steps_in, n_features)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu'), # 64 filtros de tamaño 2
    tf.keras.layers.MaxPooling1D(pool_size=2), # maxpooling size = 2
    tf.keras.layers.Flatten(), # flaten layer
    tf.keras.layers.Dense(50, activation='relu'), # dense layer
    tf.keras.layers.Dense(n_steps_out) # one output neuron
])

model2.compile(optimizer='adam', loss='mse')
model2.fit(X, y, epochs=100, verbose=False)

In [22]:
losses_df2 = pd.DataFrame(model2.history.history)
fig = px.line(losses_df2, x = losses_df1.index, y='loss', 
            title='Loss in CNN model with multiple steps')
fig.show()

In [14]:
# Generate test sample
X_test, y_test = split_sequence(test.values.flatten(), n_steps_in, n_steps_out, n_features)
print(X_test.shape, y_test.shape)
for i, val in enumerate(X_test):
    print(f"{i+1}: {val.tolist()} -> Target: {y_test[i]}")

(10, 12, 1) (10, 3)
1: [[360], [342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405]] -> Target: [417 391 419]
2: [[342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417]] -> Target: [391 419 461]
3: [[406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391]] -> Target: [419 461 472]
4: [[396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419]] -> Target: [461 472 535]
5: [[420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461]] -> Target: [472 535 622]
6: [[472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472]] -> Target: [535 622 606]
7: [[548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535]] -> Target: [622 606 508]
8: [[559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [622]] -> Target: [606 508 461]
9: [[463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [6

In [15]:
updated_array = X_test[0:1, :, :]
yhat = model2.predict(updated_array, verbose=False)
yhat

array([[401.12863, 414.6392 , 430.98096]], dtype=float32)

In [16]:
updated_array[:, 3:, :].shape, yhat.shape

((1, 9, 1), (1, 3))

In [17]:
updated_array = X_test[0:1, :, :]
list_predicted = []
limit = X_test.shape[0]
for index in range(limit):
    yhat = model2.predict(updated_array, verbose=False)  # predice el punto siguiente
    if index == limit - 1:
        my_end = yhat.flatten().tolist()
        list_predicted.extend(my_end)
        print("end predicted value:", my_end)
    else:
        list_predicted.append(yhat[0, 0].tolist())
    print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 3:, :], yhat.reshape(1, 3, 1)), axis=1)
print(f"length of list_predicted: {len(list_predicted)}")
print(list_predicted)

Predicted value: (1, 3) [[401.1286315917969, 414.6391906738281, 430.98095703125]]
Predicted value: (1, 3) [[457.23675537109375, 497.4270935058594, 541.303466796875]]
Predicted value: (1, 3) [[601.6732788085938, 557.5345458984375, 525.9202880859375]]
Predicted value: (1, 3) [[463.5365295410156, 444.7645568847656, 431.4413146972656]]
Predicted value: (1, 3) [[440.5992126464844, 452.9763488769531, 474.5220642089844]]
Predicted value: (1, 3) [[524.6293334960938, 570.1502075195312, 607.0521240234375]]
Predicted value: (1, 3) [[628.9524536132812, 591.3245849609375, 564.904296875]]
Predicted value: (1, 3) [[510.6441345214844, 485.9542541503906, 474.1505432128906]]
Predicted value: (1, 3) [[486.0848693847656, 505.45086669921875, 533.08349609375]]
end predicted value: [589.1981811523438, 626.6797485351562, 660.1609497070312]
Predicted value: (1, 3) [[589.1981811523438, 626.6797485351562, 660.1609497070312]]
length of list_predicted: 12
[401.1286315917969, 457.23675537109375, 601.6732788085938, 

In [18]:
compared2_df = test['1960-01-01':].copy()
compared2_df['Predicted'] = list_predicted
compared2_df.head(2)

,Passengers,Predicted
Month,,
1960-01-01,417,401.128632
1960-02-01,391,457.236755


In [19]:
# np.array(list_predicted).shape, y_test[:, 0].shape
rmse = np.sqrt(np.mean((compared2_df.Passengers - compared2_df.Predicted)**2))
print("RMSE:", rmse)

RMSE: 120.15253349774555


In [20]:
fig = px.line(compared2_df, x = compared2_df.index, y= ['Passengers', 'Predicted'],
            labels={'value': 'Number of passengers', 'variable': 'Legend'},
            title='Prediction with a CNN model (multi-step)')
fig.show()